In [2]:
# les données climatiques sont volumineuses (un fichier par département)
# pour ne pas les télécharger à la main, on crée un programme pour les importer automatiquement 
# en sélectionnant uniquement les variables d'intérêt

import pandas as pd
import numpy as np
from fonctions import recup_url, filtre_data

cols = ['NOM_USUEL', 
        "AAAAMM",
        "TM", 
        "TX",
        "RR", 
        "UMM", 
        "FFM", 
        "TXMIN", 
        "NBJTX0", 
        "NBJTX25", 
        "NBJTX30", 
        "NBJTX35",
        "NBJNEIG", 
        "NBJSOLNG"]

cols_indic = cols[2:len(cols)] 


# pour chaque département, on va procéder de la même façon
# on crée donc une fonction qui prend le département comme argument

def agreg_dpt(DEP):
    df_filtre = recup_url.url_to_df(url = "https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/MENS/MENSQ_" + DEP + "_previous-1950-2023.csv.gz",
                        cols_a_conserver=cols,
                        type_zip="gz",
                        plusieurs_fichiers=False)
# on crée une variable ne contenant que l'annee 
    df_filtre['AAAA'] = df_filtre['AAAAMM'].astype(str).str[:4].astype(int)
    df_filtre['MM']= df_filtre['AAAAMM'].astype(str).str[-2:].astype(int)
# on sélectionne nos mois et années d'intérêt
    df_filtre = filtre_data.filtre_annee_mois(df_filtre)
# on calcule la moyenne départementale pour toutes les variables
    df_filtre['TX_num']=df_filtre.TX.astype("float64")
    df_filtre['TXMIN_num']=df_filtre.TXMIN.astype("float64")
    df_filtre = df_filtre.groupby(['AAAA','MM'])[cols_indic].mean()
    df_filtre['DEP'] = DEP
    return(df_filtre)

# on prépare une boucle pour importer un à un les fichiers départements
# et concaténer les outputs de la fonction agreg_dpt
liste_dep = list(np.arange(2,96))
df = agreg_dpt("01")
for i in liste_dep:
    num = f'{i:02}'
    df = pd.concat([df, agreg_dpt(num)])

# on remet année et mois (devenues index) en variables normales
df = df.reset_index() 

df["DEP"] = df.DEP.astype(str)
df["DEP"] = df["DEP"].str.zfill(2)


# on crée deux departements différents pour la Corse pour pouvoir cartographier ensuite
new_rows = df.loc[df['DEP']=="20"].copy()
new_rows1 = new_rows.copy()
new_rows["DEP"] = "2A"
new_rows1["DEP"] = "2B"
df = df._append([new_rows,new_rows1])
df = df.loc[df["DEP"] != "20"]

# saison (été ou hiver)
conditions1 = [
    (df['MM'] <= 3) | (df['MM'] == 12),
    (df['MM'] >= 6) & (df['MM'] <= 9)
    ]

values1 = ['hiver', 'été']

df['saison'] = np.select(conditions1, values1, default='Other')

# période (avant ou après 2015)
conditions2 = [
    (df['AAAA'] <= 2015),
    (df['AAAA'] > 2015)
    ]

values2 = ['avant_2015', 'apres_2015']

df['periode'] = np.select(conditions2, values2, default='Other')

base_temp = df
base_temp.head()


# On veut ajouter la nouvelle base de données, que l'on vient de créer dans dossier Data
from pathlib import Path

# dossier racine du projet = 3 niveaux au-dessus de ce fichier
try:
    PROJECT_ROOT = Path(__file__).resolve().parents[2]
except NameError:
    PROJECT_ROOT = Path.cwd().parents[1]

data_dir = PROJECT_ROOT / "Data"
data_dir.mkdir(exist_ok=True)

output_path = data_dir / "data_climat.csv"
base_temp.to_csv(output_path, index=False)

print("Fichier climat sauvegardé dans :", output_path)

Fichier climat sauvegardé dans : /home/onyxia/work/Data Science/Project/Data/data_climat.csv


In [3]:
import pandas as pd
import numpy as np
from fonctions import recup_url, filtre_data

cols = [
    'ACTIVITY',
    'FREQ', 
    'GEO', 
    'GEO_OBJECT',
    'TERRTYPO',
    'TOUR_MEASURE',
    'TOUR_RESID',
    'CONF_STATUS',
    'DECIMALS',
    'OBS_STATUS',
    'OBS_STATUS_FR',
    'UNIT_MULT',
    'TIME_PERIOD',
    'OBS_VALUE'
]


#df = pd.read_csv("s3://machevallier/DS_TOUR_FREQ_data.csv", sep=';', usecols=cols)

df = recup_url.url_to_df(url = "https://www.data.gouv.fr/api/1/datasets/r/1129fd80-2564-452c-86d4-9e36e7cca4a5",
                          cols_a_conserver=cols,
                          type_zip="zip",
                          plusieurs_fichiers=True)

# on applique le facteur d'échelle 
df["OBS_VALUE_CORR"] = df["OBS_VALUE"] * (10 ** df["UNIT_MULT"])

# on met le bon nombre de décimales
df.loc[df["OBS_VALUE_CORR"].notna(), "OBS_VALUE_CORR"] = df.loc[df["OBS_VALUE_CORR"].notna()].apply(
    lambda x: round(x["OBS_VALUE_CORR"], int(x["DECIMALS"])),
    axis=1
)

# on garde seulement la valeur observée qui est corrigée
df = df.drop(columns=["DECIMALS", "UNIT_MULT", "OBS_VALUE"])

# on choisit le nombre d'arrivée comme indicateur
df = df.loc[df['TOUR_MEASURE'].isin(["ARR"])]

#df['ACTIVITY'] = df['ACTIVITY'].replace(['I553'],['CAMPING'])

#df = df[df["ACTIVITY"] == "CAMPING"]

df = df.loc[df['OBS_STATUS'].isin(["A", "P"])]
# on exclut les valeurs manquantes (O), A= Normale (définitive/validée), P= Valeur provisoire
# en faisant: print(df["OBS_STATUS"].value_counts(dropna=False))
# on obtient OBS_STATUS; A 69627; Name: count, dtype: int64
# il n'y a donc pas de P, on va pouvoir supprimer la colonne OBS_STATUS
df = df.drop(columns=["OBS_STATUS"])

# on remarque que certaines valeurs définitives sont marquées Prov sous OBS_STATUS_FR
# A = valeur correcte du point de vue technique,
# mais OBS_STATUS_FR = "PROV" = pas encore consolidée statistiquement.
# en faisant: print(df["CONF_STATUS"].value_counts(dropna=False))
# on obtient CONF_STATUS; F 69627; Name: count, dtype: int64
# il n'y a donc que des observations diffusables, on va pouvoir supprimer la colonne CONF_STATUS
df = df.drop(columns=["CONF_STATUS","TOUR_MEASURE","OBS_STATUS_FR"])

# On prend les données mensuelles et on supprime la colonne FREQ
df = df.loc[df['FREQ'].isin(["M"])]
df = df.drop("FREQ", axis = 1)

# On définit les années que l'on veut garder
# on crée une variable ne contenant que l'annee et une autre le mois
df['AAAA'] = df['TIME_PERIOD'].astype(str).str[:4].astype(int)
df['MM']= df['TIME_PERIOD'].astype(str).str[5:7].astype(int)
df = df.drop("TIME_PERIOD", axis = 1)

#on filtre les données sur nos mois d'interet
df = filtre_data.filtre_annee_mois(df)

# On prend les données de departement et on supprime c
df = df.loc[df['GEO_OBJECT'].isin(["DEP"])]
df = df.drop("GEO_OBJECT", axis = 1)

# on supprime TERRTYPO car tout est identique
df = df.drop("TERRTYPO", axis = 1)

# on exclut les DOM TOM
df = df.loc[df['GEO'].str.len() == 2]

DEP_NOM = {
    "01": "Ain", "02": "Aisne", "03": "Allier", "04": "Alpes-de-Haute-Provence",
    "05": "Hautes-Alpes", "06": "Alpes-Maritimes", "07": "Ardèche", "08": "Ardennes",
    "09": "Ariège", "10": "Aube", "11": "Aude", "12": "Aveyron", "13": "Bouches-du-Rhône",
    "14": "Calvados", "15": "Cantal", "16": "Charente", "17": "Charente-Maritime",
    "18": "Cher", "19": "Corrèze", "21": "Côte-d'Or", "22": "Côtes-d'Armor",
    "23": "Creuse", "24": "Dordogne", "25": "Doubs", "26": "Drôme", "27": "Eure",
    "28": "Eure-et-Loir", "29": "Finistère", "2A": "Corse", "2B": "Corse",
    "30": "Gard", "31": "Haute-Garonne", "32": "Gers", "33": "Gironde", "34": "Hérault",
    "35": "Ille-et-Vilaine", "36": "Indre", "37": "Indre-et-Loire", "38": "Isère",
    "39": "Jura", "40": "Landes", "41": "Loir-et-Cher", "42": "Loire", "43": "Haute-Loire",
    "44": "Loire-Atlantique", "45": "Loiret", "46": "Lot", "47": "Lot-et-Garonne",
    "48": "Lozère", "49": "Maine-et-Loire", "50": "Manche", "51": "Marne",
    "52": "Haute-Marne", "53": "Mayenne", "54": "Meurthe-et-Moselle", "55": "Meuse",
    "56": "Morbihan", "57": "Moselle", "58": "Nièvre", "59": "Nord", "60": "Oise",
    "61": "Orne", "62": "Pas-de-Calais", "63": "Puy-de-Dôme", "64": "Pyrénées-Atlantiques",
    "65": "Hautes-Pyrénées", "66": "Pyrénées-Orientales", "67": "Bas-Rhin",
    "68": "Haut-Rhin", "69": "Rhône", "70": "Haute-Saône", "71": "Saône-et-Loire",
    "72": "Sarthe", "73": "Savoie", "74": "Haute-Savoie", "75": "Paris",
    "76": "Seine-Maritime", "77": "Seine-et-Marne", "78": "Yvelines", "79": "Deux-Sèvres",
    "80": "Somme", "81": "Tarn", "82": "Tarn-et-Garonne", "83": "Var", "84": "Vaucluse",
    "85": "Vendée", "86": "Vienne", "87": "Haute-Vienne", "88": "Vosges", "89": "Yonne",
    "90": "Territoire de Belfort", "91": "Essonne", "92": "Hauts-de-Seine",
    "93": "Seine-Saint-Denis", "94": "Val-de-Marne", "95": "Val-d'Oise",
    "971": "Guadeloupe", "972": "Martinique", "973": "Guyane", "974": "La Réunion",
    "976": "Mayotte"
}

df.insert(
    1,  # position (0 = première colonne, 1 = deuxième, etc.)
    "DEP_NOM",  # nom de la nouvelle colonne
    df["GEO"].map(DEP_NOM)
)

#changer le nom de la colonne GEO en DEP pour la fusion
col = df.columns.tolist()
col[0] = "DEP"
df.columns = col

df['DEP'] = df['DEP'].replace(['2A', '2B'],['20', '20'])

# la variable "TOUR_RESID" donne l'origine du touriste : on filtre sur total (on ne distingue pas pour l'instant)
df = df[df["TOUR_RESID"] != "_T"]

# print(df["ACTIVITY"].value_counts(dropna=False))
# Aucun camping n'est présent dans notre sélection

df['TOUR_RESID'] = df['TOUR_RESID'].replace(['250', '1_X_250'],['France', 'Étranger'])

# on somme les arrivées par année et mois
df = df.groupby(['AAAA','MM', 'DEP', 'DEP_NOM', 'TOUR_RESID'])["OBS_VALUE_CORR"].sum()

# on remet année et mois (devenues index) en variables normales
df = df.reset_index() 

base_touri = df

# On veut mettre notre nouvelle base de données dans Data 
from pathlib import Path

try:
    PROJECT_ROOT = Path(__file__).resolve().parents[2]
except NameError:
    PROJECT_ROOT = Path.cwd().parents[1]
data_dir = PROJECT_ROOT / "Data"
data_dir.mkdir(exist_ok=True)

output_path = data_dir / "data_tourisme.csv"
base_touri.to_csv(output_path, index=False)

print("Fichier tourisme sauvegardé dans :", output_path)




Fichier tourisme sauvegardé dans : /home/onyxia/work/Data Science/Project/Data/data_tourisme.csv
